In [9]:
import os

from utils import load_jsonl_iteratively 

lang = "ja"
train_data_root = "/data/xzhao/experiments/med-eval/dataset/losses/train_samples"
data_path = os.path.join(train_data_root, f"random_{lang}_2k.jsonl")

for item in load_jsonl_iteratively(data_path):
    print(item["text"])
    break

次の文章に続きの文を加えてください。
我々は, 細菌感染によってウサギに慢性副鼻腔炎を惹起することにより, 副鼻腔及び鼻腔にポリープが形成されるのを認めた。...
これらは組織学的特徴から,「肉芽性ポリープ」および「浮腫性ポリープ」の二つの型に分けられた。 「肉芽性ポリープ」は上皮を完全に失った部で, 線維芽細胞の増殖と血管新生が起こって肉芽が洞内に突出することによって形成され, それを被うように未分化な上皮細胞が増殖移動していくように観察された。 「浮腫性ポリープ」は, 上皮細胞が脱落したり, 成熟した形態を失って背の低い未分化な形態を示したり, 扁平な上皮細胞が重層して観察されたりする部で, 形成されていた。 本研究の結果は, 上皮細胞も鼻茸形成の初期において, なんらかの役割を果たしていることを示唆する。


In [10]:
import csv
import random
import sqlite3

conn = sqlite3.connect("resources/wnjpn.db")
def csv_input(path_name):
    rows = []
    with open(path_name,encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            rows.append(row)
    return rows


def SearchSimilarWords(word, mode="simple"):
    assert mode in ["simple", "detailed"], "mode must be 'simple' or 'detailed'."
    cur = conn.execute("select wordid from word where lemma='%s'" % word)
    word_id = 99999999 
    for row in cur:
        word_id = row[0]

    if word_id==99999999:
        return
    cur = conn.execute("select synset from sense where wordid='%s'" % word_id)
    synsets = []
    for row in cur:
        synsets.append(row[0])
    simdict = {}

    en_syns, ja_syns = [], []
    for synset in synsets:
        # cur1 = conn.execute("select name from synset where synset='%s'" % synset)
        cur2 = conn.execute("select def from synset_def where (synset='%s' and lang='jpn')" % synset)
        cur3 = conn.execute("select wordid from sense where (synset='%s' and wordid!=%s)" % (synset,word_id))
            
        if mode == "simple":
            for row3 in cur3:
                target_word_id = row3[0]
                cur3_1 = conn.execute("select lemma, lang from word where wordid=%s" % target_word_id)
                for row3_1 in cur3_1:
                    if row3_1[1] == "jpn":
                        ja_syns.append(row3_1[0])
                    elif row3_1[1] == "eng":
                        if "_" in row3_1[0]:
                            en_syns.append(row3_1[0].replace("_", " "))
                        else:
                            en_syns.append(row3_1[0])
                    else:
                        raise ValueError("Unknown language: %s" % row3_1[1])
        elif mode == "detailed":
            simdict[synset] = {"definition": "", "words": []}
            simdict[synset]["definition"] = ". ".join([row[0] for row in cur2])
            for row3 in cur3:
                target_word_id = row3[0]
                cur3_1 = conn.execute("select lemma, lang from word where wordid=%s" % target_word_id)
                for row3_1 in cur3_1:
                    simdict[synset]["words"].append(row3_1)

    if mode == "simple":
        simdict["ja"] = random.choice(ja_syns) if ja_syns else None
        simdict["en"] = random.choice(en_syns) if en_syns else None
        if simdict["ja"] == None and simdict["en"] == None:
            return None
    return simdict

In [11]:
# Mecabなど形態素解析で使うIPA品詞体系 https://miner.hatenablog.com/entry/323
from sudachipy import tokenizer
from sudachipy import dictionary
import stopwordsiso as stopwords

japanese_stopwords = stopwords.stopwords("ja")
english_stop_words = stopwords.stopwords("en")
stopwords = japanese_stopwords.union(english_stop_words)

tokenizer_obj = dictionary.Dictionary(dict_type="full").create()
mode = tokenizer.Tokenizer.SplitMode.C

def process_syn(text):
    text = item["text"]
    tokens = tokenizer_obj.tokenize(text, mode)

    words = [(m.surface(), m.part_of_speech(), (m.begin(), m.end())) for m in tokens]
    syns = []
    
    def get_word_syn(word, pos, index):
        try:
            syns = SearchSimilarWords(word)
        except Exception as e:
            return None
        if not syns:
            return None
        
        return {
            "surface": word,
            "pos": pos,
            "index": index,
            "synonyms": syns if syns else []
        }

    for word in words:
        if len(word[0]) < 2 or word[0] in stopwords:
            continue
        
        if (word[1][0] == "名詞" and word[1][1] in ["普通名詞", "固有名詞", "サ変接続", "形容動詞語幹"]) or \
            (word[1][0] == "動詞" and word[1][1] == "一般"):
            syn_item = get_word_syn(word[0], word[1], word[2])
            if syn_item:
                syns.append(syn_item)
    
    all_words = [m.surface() for m in tokens]
    # print(len(syns) / len(all_words))
    return {"syn_words": syns, "all_words": all_words}

/tmp/ipykernel_3882146/2287453364.py:10: DeprecationWarning: Parameter dict_type of Dictionary() is deprecated, use dict instead
  tokenizer_obj = dictionary.Dictionary(dict_type="full").create()


In [12]:
from tqdm import tqdm
from utils import dump_jsonl

item_with_syns = []
for item in tqdm(load_jsonl_iteratively(data_path, request_num=10)):
    item['wordnet_synonyms'] = process_syn(item["text"])
    item_with_syns.append(item)

# dump_path = os.path.join(train_data_root, f"random_{lang}_2k.withsyn.jsonl")
# dump_jsonl(item_with_syns, dump_path)

10it [00:02,  4.22it/s]


TEST from here

In [20]:
from transformers import AutoTokenizer

llm_tokenizer = AutoTokenizer.from_pretrained("/data/xzhao/experiments/roman-pretrain/exps/exp1-multi-ja/hf_models/iter_0000010")

/home/xzhao/.pyenv/versions/poetry-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
import nltk
from nltk.metrics.distance import edit_distance

seq1 = ["人間", "の", "体", "は", "約", "37兆個", "の", "細胞", "で", "構成", "さ", "れ", "て", "いる"]
seq2 = ["人間", "の", "体", "は", "約", "37兆", "の", "DNA細胞", "から", "作ら", "れて", "いる", "x"]

distance = edit_distance(seq1, seq2)
print("Levenshtein distance:", distance)

Levenshtein distance: 8


In [114]:
idx = 5
text = item_with_syns[idx]["text"]
synonyms = item_with_syns[idx]["wordnet_synonyms"]["syn_words"]

In [128]:
def get_synonym_text(synonyms, tgt_lang, prob):
    replaced_syns = []
    noisy_text = text
    synonyms = [syn for syn in synonyms if syn["synonyms"][tgt_lang] is not None]
    for syn in synonyms:
        if random.random() < prob: 
            replaced_syns.append(syn)

    replaced_indexes = []
    offset = 0
    for synonym in replaced_syns:
        replaced_indexes.append((synonym["index"][0] + offset, synonym["index"][0] + len(synonym["synonyms"][tgt_lang]) + offset))
        offset += len(synonym["synonyms"][tgt_lang]) - (synonym["index"][1] - synonym["index"][0])

    replaced_syns = sorted(replaced_syns, key=lambda x: x["index"][0], reverse=True)
    for synonym in replaced_syns:
        noisy_text = noisy_text[:synonym["index"][0]] + synonym["synonyms"][tgt_lang] + noisy_text[synonym["index"][1]:]
    return noisy_text, replaced_syns[::-1], replaced_indexes

In [127]:
noisy_text, replaced_syns, replaced_indexes = get_synonym_text(synonyms, "ja", 0.5)
for syn, index in zip(replaced_syns, replaced_indexes):
    print(f"Replaced '{syn['surface']}' by '{syn['synonyms']['ja']}', '{noisy_text[index[0]:index[1]]}' at index {index[0]}-{index[1]}")

Replaced '提供' by '供与', '供与' at index 6-8
Replaced '使用' by '使う', '使う' at index 72-74
Replaced '場合' by '時点', '時点' at index 77-79
Replaced '肺がん' by '肺癌', '肺癌' at index 119-121
Replaced '治療' by '和らげる', '和らげる' at index 122-126
Replaced 'がん' by '癌腫', '癌腫' at index 152-154
Replaced '防ぎ' by '防護具', '防護具' at index 160-163


In [155]:
noisy_text, replaced_syns, replaced_indexes = get_synonym_text(synonyms, "ja", 0.5)
tokens = llm_tokenizer(text)["input_ids"]
noisy_tokens = llm_tokenizer(noisy_text, return_offsets_mapping=True)
noisy_token_positions = noisy_tokens["offset_mapping"]
noisy_tokens = noisy_tokens["input_ids"]
print(text)
print("----------")
print(noisy_text)


適切な答えを提供してください。パーキチン（paclitaxel）とカルボプラチン（carboplatin）の組み合わせは、どのようながんの治療に使用される場合がありますか？
パーキチンとカルボプラチンの組み合わせは、特に卵巣がん、乳がん、肺がんの治療に用いられます。これらの薬はしばしばともに使われて、がん細胞の分裂を防ぎ、それに適度なストレスをかけて治療効果を高めます。
----------
適切な答えを供与してください。パーキチン（paclitaxel）とカルボプラチン（carboplatin）の組み合わせは、どのような癌の緩和に使用される場合がありますか？
パーキチンとカルボプラチンの組み合わせは、特に卵巣がん、乳がん、肺癌の治療に用いられます。これらの薬はしばしばともに使われて、癌腫細胞の分かれるを防護具、それに適度なストレスをかけて治療効果を高めます。


In [147]:
noisy_tokens = llm_tokenizer(noisy_text, return_offsets_mapping=True)

In [150]:
replaced_indexes

[(54, 58), (71, 73), (100, 104), (151, 153), (154, 158)]

In [156]:
token_labels = []
for idx, (start, end) in enumerate(noisy_token_positions):
    for index in replaced_indexes:
        if start >= index[0] and end <= index[1]:
            token_labels.append("<synonym>")
            break
    if len(token_labels) <= idx:
        token_labels.append("<token>")
    